<a href="https://colab.research.google.com/github/Gruppe3VDL/3D_computer_vision/blob/master/CV_without_project_and_draw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import os, sys
import numpy as np
import cv2 as cv
import scipy.io as io
def project_points(X, K, R, T, distortion_flag=False, distortion_params=None):
    """Project points from 3d world coordinates to 2d image coordinates.
    Your code should work with considering distortion and without
    considering distortion parameters.
    """
    # Your implementation goes here!
    k1, k2, k5 = distortion_params[0], distortion_params[1], distortion_params[4]
    Camera_2D_Points = []
    print(k1)
    print(k2)
    print(k5)
    for i in range(X.shape[0]):
      #40x3x1
      x_3d_point=X[i].reshape(X.shape[1],3,1)
      Extrinsix_Params=np.matmul(R[i],x_3d_point)+T[i]
      Intrinsix_Params=np.matmul(K,Extrinsix_Params)
      Intrinsix_Params[:, 0, :]=Intrinsix_Params[:, 0, :]/Intrinsix_Params[:, 2, :]
      Intrinsix_Params[:, 1, :]=Intrinsix_Params[:, 1, :]/Intrinsix_Params[:, 2, :]
      Intrinsix_Params[:, 2, :]=Intrinsix_Params[:, 2, :]/Intrinsix_Params[:, 2, :]
      #print("Intrinsic Parameters",Intrinsix_Params)
      if distortion_flag==True:
          Inverse_K = np.linalg.inv(K)
          Normalized_Matrix = np.matmul(Inverse_K, Intrinsix_Params)
          Normalized_Matrix = Normalized_Matrix[:, 0:2, 0]
          R2 = Normalized_Matrix[:, 0]**2 + Normalized_Matrix[:, 1]**2
          D_Parameter = 1 + k1*R2 + k2*(R2**2) + k5*(R2**3)
          a2D_Camera_x = Normalized_Matrix[:,0]*D_Parameter
          a2D_Camera_y = Normalized_Matrix[:,1]*D_Parameter
          a2D_Camera_x = a2D_Camera_x.reshape(X.shape[1], 1)
          a2D_Camera_y = a2D_Camera_y.reshape(X.shape[1], 1)
          Homogenious_Matrix2D_Camera = np.concatenate((a2D_Camera_x, a2D_Camera_y, np.ones(a2D_Camera_x.shape)), axis=1)
          Matrix2D_Camera_C_TO_POINT = np.matmul(K, Homogenious_Matrix2D_Camera.reshape(X.shape[1], 3, 1))
          Camera_2D_Points.append(Matrix2D_Camera_C_TO_POINT[:, :-1, 0])
    if distortion_flag==True:
      return [Camera_2D_Points]     
    else:
      Print("No distorsion Found")
      return [Intrinsix_Params[:, :-1, 0]]


      '''r_x=[]
      r_y=[]
      for i, item in enumerate(Intrinsic_Params_Norm):
        #print(i)

        x_item=item[0][0]*item[0][0]
        y_item=item[0][1]*item[0][1]
        r_square=x_item+y_item
        #result=math.pow( sum_x_y, 2 )
        #1+k1*r_2+k2*r_4
        r_4=math.pow( r_square, 2)
        equation=1+k1*r_square+k2*r_4
        equation_x=equation*item[0][0]
        equation_y=equation*item[0][1]
        r_x.append(equation_x)
        r_y.append(equation_y)
        
        #print("R",r_square)
      print("R_X",r_x[1])
      print("R_Y",r_y[1])
        #radial_distorion=ite
      break
   '''

    #print("Extrinsic Parameters Matrix",projection)
    #raise NotImplementedError

def project_and_draw(img, X_3d, K, R, T, distortion_flag, distortion_parameters):
    """
    call "project_points" function to project 3D points to camera coordinates
    draw the projected points on the image and save your output image here
    # save your results in a separate folder named "results"
    # Your implementation goes here!

    """
    total_points = X_3d.shape[1]
    result_2d = project_points(X_3d, K, R, T, distortion_flag, distortion_parameters)
  

if __name__ == '__main__':
    base_folder = '/content/drive/My Drive/data/'

    # Consider distorition
    dist_flag = True

    # Load the data
    # There are 25 views/or images/ and 40 3D points per view
    data = io.loadmat('ex_1_data.mat')
    
    # 3D points in the world coordinate system
    X_3D = data['x_3d_w'] # shape=[25, 40, 3]

    # Translation vector: as the world origin is seen from the camera coordinates
    TVecs = data['translation_vecs'] # shape=[25, 3, 1]
    
    # Rotation matrices: project from world to camera coordinate frame
    RMats = data['rot_mats'] # shape=[25, 3, 3]

    # five distortion parameters
    dist_params = data['distortion_params']

    # K matrix of the cameras
    Kintr = data['k_mat'] # shape 3,3
    
    imgs_list = [cv.imread(base_folder+str(i).zfill(5)+'.jpg') for i in range(TVecs.shape[0])]
    imgs = np.asarray(imgs_list)
    '''X_3D_H=np.empty([25, 40,4])
    imgs = np.asarray(imgs_list)
    for i, item in enumerate(X_3D):
      for j, item in enumerate(item):
        X_3D_H[i][j]=np.append(X_3D[i][j], 1)'''
    

    project_and_draw(imgs, X_3D, Kintr, RMats, TVecs, dist_flag, dist_params)

[-0.30746699]
[0.10299779]
[-0.01612529]
[[[ 74  82  59]
  [ 62  91  41]
  [124 190  89]
  ...
  [ 55  83  83]
  [ 54  84  85]
  [ 54  84  85]]

 [[124 145 123]
  [119 156 116]
  [111 178 103]
  ...
  [ 55  83  83]
  [ 53  83  84]
  [ 54  84  85]]

 [[119 169 151]
  [121 177 158]
  [125 194 174]
  ...
  [ 55  83  83]
  [ 53  83  84]
  [ 53  83  84]]

 ...

 [[ 23  24  22]
  [ 22  23  21]
  [ 21  22  20]
  ...
  [ 25  30  31]
  [ 25  30  33]
  [ 25  30  33]]

 [[ 23  24  22]
  [ 22  23  21]
  [ 21  22  20]
  ...
  [ 24  29  30]
  [ 25  30  33]
  [ 25  30  33]]

 [[ 20  21  19]
  [ 20  21  19]
  [ 20  21  19]
  ...
  [ 24  29  30]
  [ 25  30  33]
  [ 25  30  33]]]
[[[ 85 111  75]
  [ 93 112  61]
  [ 53  57   0]
  ...
  [ 56  84  85]
  [ 59  87  88]
  [ 59  87  88]]

 [[121 158 132]
  [131 164 127]
  [120 141  79]
  ...
  [ 57  85  86]
  [ 58  86  87]
  [ 58  86  87]]

 [[120 180 174]
  [122 180 169]
  [129 182 162]
  ...
  [ 57  85  86]
  [ 58  86  87]
  [ 60  88  89]]

 ...

 [[ 23  24 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!wget https://github.com/Gruppe3VDL/3D_computer_vision/blob/master/ex_1_data.mat?raw=true
# rename the file
!mv ex_1_data.mat\?raw\=true ex_1_data.mat

--2019-11-16 23:57:12--  https://github.com/Gruppe3VDL/3D_computer_vision/blob/master/ex_1_data.mat?raw=true
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/Gruppe3VDL/3D_computer_vision/raw/master/ex_1_data.mat [following]
--2019-11-16 23:57:12--  https://github.com/Gruppe3VDL/3D_computer_vision/raw/master/ex_1_data.mat
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Gruppe3VDL/3D_computer_vision/master/ex_1_data.mat [following]
--2019-11-16 23:57:12--  https://raw.githubusercontent.com/Gruppe3VDL/3D_computer_vision/master/ex_1_data.mat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... con